In [1]:
import atlite
import linkingtool.linking_utility as utils
import linkingtool.linking_data as dataprep
import geopandas as gpd
import os
import osmnx as ox
import geopandas as gpd
import pandas as pd

In [26]:
class OSM_power_network_processor:
    def __init__(self, area_name: str, area_short_code: str):
        self.area = area_name
        self.area_sc = area_short_code
        # Define the OSM tags for power infrastructure
        self.tags = {'power': ['line','cable','minor_line', 'substation', 'tower', 'pole', 'generator','plant','terminal'],
                     'substation':['transmission','distribution','minor_distribution','industrial']
                     }

    def load_power_network_from_OSM(self):
        """
        Retrieve all power-related infrastructure within the specified area."""
        # Retrieve the power network data from OSM
        self.power_network = ox.features_from_place(self.area, self.tags)

    def save_local_file(self):
        """
        Save the power network data to a GeoJSON file (optional).
        """
        geojson_path = f"{self.area_sc}.geojson"  # Ensure file name is properly formatted
        self.power_network.to_file(geojson_path, driver='GeoJSON')

In [30]:
# Example usage:
processor = OSM_power_network_processor("Surrey,British Columbia, Canada","Sur_BC")
processor.load_power_network_from_OSM()
processor.save_local_file()


In [ ]:
# Example of using 'CartoDB positron' basemap
xx.explore('power', tiles='CartoDB positron')


In [2]:
config=utils.load_config('config/config_linking_tool.yml')

In [3]:
province_gadm_regions_gdf=gpd.read_file('data/downloaded_data/GADM/BC_gadm_regions.geojson')
current_region=config['regional_info']['region_1']
_CRC_=current_region['code']
cutout_config=config['cutout']

In [ ]:
bounding_box_save_to:str=os.path.join(config['cutout']['root'],f"{_CRC_}_MBR.geojson")
bounding_box:dict=dataprep.plot_n_save_bounding_box(province_gadm_regions_gdf,current_region,bounding_box_save_to)

In [7]:
cutout_config = {
    'root': 'data/downloaded_data/cutout',
    'module': ['era5'],
    'dx': 0.25,
    'dy': 0.25,
    'snapshots': {
        'start': ['2023-01-01 07:00:00'],
        'end': ['2023-01-01 11:00:00']
    },
    'snapshots_tz_BC': {
        'start': ['2023-01-01 00:00:00'],
        'end': ['2023-01-01 04:00:00']
    },
    'timezone_convert': 'Etc/GMT+7'
}



In [10]:
def prepare_cutout(module,start_date, end_date,bounding_box,save_to_path):

    min_x,max_x,min_y,max_y=bounding_box.values()

    cutout = atlite.Cutout(
        path=save_to_path,
        module=module,
        x=slice(min_x,max_x), # Longitude
        y=slice(min_y,max_y), # Latitude
        time=slice(start_date,end_date)
    )

    cutout.prepare()
    return True

In [11]:
start_date="2023-01-01"
end_date="2023-01-01"

In [ ]:
import atlite

# values are random, any cutout will fail
cutout = atlite.Cutout(
    path='test.nc',
    module=['era5'],
    x=slice(3, 6),
    y=slice(3, 6),
    time="2019-05-02",
    features=['runoff'],
)
cutout.prepare()

In [ ]:
dataprep.create_era5_cutout('BC_test_',bounding_box,cutout_config)

In [5]:
import atlite
import xarray as xr

In [2]:
file_path='data/downloaded_data/cutout/era5_data_BC_2022.nc'

In [8]:
ds=xr.open_dataset(file_path, chunks={})

In [ ]:
ds.rename({'ssr': 'influx_toa'})

In [ ]:
ds.rename({'v100': 'wnd100m'})


In [10]:
cutout=atlite.Cutout('data/downloaded_data/cutout/BC_2022.nc')

In [ ]:
cutout.data

In [11]:
gwa_cells=pl.DataFrame ('data/Processed_data/Wind/BC/gwa_cells.pkl')

In [ ]:
gwa_cells